# Задание 1
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание 2
* Выберите регион
* Сравните среднюю цену квадратного метра в городах этого региона
* Определите количество строящихся объектов и квартир в регионах
* Определите объем строящегося жилья в квадратных метрах

# Задание 3 *
* Напишете скрипт, который сможет ежедневно собирать данные по цене, количестве и объему строящихся объектов самостоятельно и ежедневно
* Попробуйте ускорить процесс
* Добавьте возможность добавления данных в БД с указанием текущей даты.

In [42]:
import requests
import re
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import sqlite3

## Список объектов

In [3]:
# offset_ = 2001
# limit_ = 1000
# url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
# response = requests.get(url)
# objects_data = response.json() 

In [6]:
# objects_list = objects_data.get('data').get('list')

In [7]:
# objids = [x.get('objId') for x in objects_list]

In [16]:
# len(objids)

In [3]:
limit_ = 1000
all_obj_ids = []
for offset_ in tqdm(range(0, 12000, 1000)):
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    response = requests.get(url)
    objects_data = response.json()
    objects_list = objects_data.get('data').get('list')
    objids = [x.get('objId') for x in objects_list]
    all_obj_ids.extend(objids)
    
print("Список ID объектов собран")

  0%|          | 0/12 [00:00<?, ?it/s]

Список ID объектов собран


In [23]:
len(all_obj_ids)

10076

# Данные по определенному объекту

In [4]:
all_objects_description = []

for id in tqdm(all_obj_ids):
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{id}'
    response_object = requests.get(url)
    obj = response_object.json()
    all_objects_description.append(obj)

print('Список объектов собран')

  0%|          | 0/10076 [00:00<?, ?it/s]

Список объектов собран


In [72]:
all_objects_description[5]

{'data': {'id': 23682,
  'pdId': 64486,
  'developer': {'devId': 6811,
   'devShortCleanNm': '84 ВЫСОТА',
   'devShortNm': '84 ВЫСОТА',
   'devFullCleanNm': '84 ВЫСОТА',
   'problObjCnt': 0,
   'buildObjCnt': 1,
   'comissObjCnt': 6,
   'regRegionDesc': 'Санкт-Петербург город',
   'devPhoneNum': '+7(812)309-84-09',
   'devSite': 'www.84visota.ru',
   'devEmail': '84visota@mail.ru',
   'devInn': '7806536079',
   'devOgrn': '1147847382627',
   'devKpp': '780601001',
   'devLegalAddr': 'город Санкт-Петербург, улица Белорусская дом 12 корпус 1 литера А помещение 1Н',
   'devFactAddr': 'г Санкт-Петербург ул. Белорусская д. 12 кор. 1 стр. лит. А помещение 1Н',
   'lastRpdId': 529403,
   'fundGuarantyFlg': 1,
   'devOrgRegRegionCd': 78,
   'devEmplMainFullNm': 'Искрицкий Максим Николаевич',
   'developerGroupName': '84 Высота',
   'orgForm': {'id': 4,
    'fullForm': 'Общество с ограниченной ответственностью',
    'shortForm': 'ООО'},
   'companyGroupId': 5562608001,
   'objGuarantyEscrowFlg'

In [61]:
# преобразовали список 'all_objects_description' в DataFrame

df = pd.DataFrame.from_dict(pd.json_normalize(all_objects_description), orient='columns')

In [62]:
df.head()

,errcode,data.id,data.pdId,data.developer.devId,data.developer.devShortCleanNm,data.developer.devShortNm,data.developer.devFullCleanNm,data.developer.problObjCnt,data.developer.buildObjCnt,data.developer.comissObjCnt,...,data.metro.colors,data.transportDistIndex,data.transportDistIndexValue,data.greenAreaIndexValue,data.airQualityIndexValue,data.complexShortNm,data.developer.bankruptStage.bankruptStageCd,data.developer.bankruptStage.bankruptStageDesc,data.developer.bankruptStage.bankruptLawUrl,data.developer.orgBankruptMsgDttm
0,0,13391,28101,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,13383,28105,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,13397,28141,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,13398,28141,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,13399,28141,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10076 entries, 0 to 10075
Columns: 109 entries, errcode to data.developer.orgBankruptMsgDttm
dtypes: bool(2), float64(26), int64(30), object(51)
memory usage: 8.2+ MB


In [15]:
# Cохранить pandas dataframe в excel

writer = pd.ExcelWriter("houses_under_construction.xlsx", engine = 'xlsxwriter')
df.to_excel(writer)
writer.save()
print('DataFrame успешно сохранен в Excel файл')

DataFrame успешно сохранен в Excel файл


In [24]:
# pd.read_excel("houses_under_construction.xlsx", index_col=0)

In [16]:
# Cохранить pandas dataframe в pickle

df.to_pickle('houses_under_construction.pkl')

print('DataFrame успешно сохранен в pickle файл')

DataFrame успешно сохранен в pickle файл


In [2]:
# pd.read_pickle('houses_under_construction.pkl')

In [41]:
# скопируем DataFrame, чтобы удалить столбцы и записать в БД
df_1 = df.copy()

In [68]:
df_1.drop(columns=['data.photoRenderDTO', 'data.objectTransportInfo', 'data.metro.colors'], inplace=True)
df_1.head()

,errcode,data.id,data.pdId,data.developer.devId,data.developer.devShortCleanNm,data.developer.devShortNm,data.developer.devFullCleanNm,data.developer.problObjCnt,data.developer.buildObjCnt,data.developer.comissObjCnt,...,data.metro.isWalk,data.transportDistIndex,data.transportDistIndexValue,data.greenAreaIndexValue,data.airQualityIndexValue,data.complexShortNm,data.developer.bankruptStage.bankruptStageCd,data.developer.bankruptStage.bankruptStageDesc,data.developer.bankruptStage.bankruptLawUrl,data.developer.orgBankruptMsgDttm
0,0,13391,28101,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,13383,28105,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,13397,28141,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,13398,28141,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,13399,28141,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
# Cохранить pandas dataframe в БД
from sqlalchemy import create_engine
engine = create_engine(r'sqlite:///C:\Users\anna_\Downloads\DaPython.db')  # создаем соединение

In [56]:
df_1.to_sql(name='houses_under_construction', con=engine)

print('DataFrame успешно сохранен в SQLite БД')

DataFrame успешно сохранен в SQLite БД


In [64]:
# engine.execute("SELECT * FROM houses_under_construction").fetchmany(2)

[(0, '0', 13391, 28101, 306, '2МЕН ГРУПП', '2МЕН ГРУПП', '2МЕН ГРУПП', 0, 5, 0, 'Тюменская область', '+7(345)279-18-88', '2mengroup.ru', 'info@2mengroup.ru', '7701651356', '1067746424899', '720301001', 'Тюменская область, город Тюмень, улица Комсомольская дом 75/5', 'Тюменская область, город Тюмень, улица Комсомольская 75/5', 585074, 1, 72.0, 'Киселев Сергей Михайлович', '2МЕН ГРУПП ДЕВЕЛОПМЕНТ', 1, 'Акционерное общество', 'АО', 699421001.0, 0, 0, 72, 'г Тюмень, ул Профсоюзная', 3, 12, 44, '2022-06-30', 'Другое', 'По желанию заказчика', 'Есть', 41, 4293.67, '72-000096', 'https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/014fb813-6b4d-4377-8781-78b566f1f9ab', '29-04-2022 13:16', '57.1541', '65.5595', 1, 0, 'Элитный', '03-05-2022 01:10', 0, 'normal', 'https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/09ff8156-bcbb-4033-a36a-dd45abd0b430', 1, 37219, 12, 44, '4293.67', 26, 11.0, 16.0, 12.0, 5.0, 44.0, 2.0, 4.0, 0, 1, '30-08-2022 12:00', '2,8', 0, 1.0, 1.0, 1.0, 0.0, 9.0, 1, 1, 0.0, 2.0, 

In [60]:
# pd.read_sql('select * from houses_under_construction', con=engine)

## Задание 2

In [16]:
# создадим копию DataFrame, чтобы выбрать необходимые столбцы и переименовать их
data = df[['data.id', 'data.developer.regRegionDesc', 'data.developer.devFactAddr', 'data.developer.devShortNm', 'data.objSquareLiving', 'data.objFlatCnt', 'data.objFloorCnt', 'data.objPriceAvg']]
data.rename(columns={'data.id': 'ID дома', 'data.developer.regRegionDesc': 'Регион', 
                     'data.developer.devFactAddr': 'Адрес дома', 'data.developer.devShortNm':'Застройщик', 
                     'data.objSquareLiving': 'Жилая площадь', 'data.objFlatCnt': 'Кол-во квартир', 
                     'data.objFloorCnt': 'Кол-во этажей', 'data.objPriceAvg': 'Цена за 1 кв.м'}, inplace=True)
data.head()

C:\Users\anna_\anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,ID дома,Регион,Адрес дома,Застройщик,Жилая площадь,Кол-во квартир,Кол-во этажей,Цена за 1 кв.м
0,13391,Тюменская область,"Тюменская область, город Тюмень, улица Комсомо...",2МЕН ГРУПП,4293.67,44,12,55893.0
1,13383,Тюменская область,"Тюменская область, город Тюмень, улица Комсомо...",2МЕН ГРУПП,6289.98,68,20,53109.0
2,13397,Тюменская область,"Тюменская область, город Тюмень, улица Комсомо...",2МЕН ГРУПП,6248.51,65,12,45523.0
3,13398,Тюменская область,"Тюменская область, город Тюмень, улица Комсомо...",2МЕН ГРУПП,4812.98,43,10,47886.0
4,13399,Тюменская область,"Тюменская область, город Тюмень, улица Комсомо...",2МЕН ГРУПП,0.00,0,1,50768.0


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10076 entries, 0 to 10075
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID дома         10076 non-null  int64  
 1   Регион          9968 non-null   object 
 2   Адрес дома      10067 non-null  object 
 3   Застройщик      10076 non-null  object 
 4   Жилая площадь   10076 non-null  float64
 5   Кол-во квартир  10076 non-null  int64  
 6   Кол-во этажей   10076 non-null  int64  
 7   Цена за 1 кв.м  7609 non-null   float64
dtypes: float64(2), int64(3), object(3)
memory usage: 629.9+ KB


In [79]:
# Используя регулярные выражения, извлечем населенный пункт и добавим его в соответствующий столбец,
# чтобы потом сгруппировать по городам

str_pat = r'(город|г|поселок|п|поселок городского типа|пгт|село|с) ([а-яА-Я-]*)( [а-я]{1}|, [а-я]{1})'
data['Населенный пункт'] = data['Адрес дома'].str.extract(str_pat, 0)[1]
data[data['Регион'] == 'Приморский край']

C:\Users\anna_\AppData\Local\Temp/ipykernel_15036/159690340.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Населенный пункт'] = data['Адрес дома'].str.extract(str_pat, 0)[1]


,ID дома,Регион,Адрес дома,Застройщик,Жилая площадь,Кол-во квартир,Кол-во этажей,Цена за 1 кв.м,Населенный пункт
6,36321,Приморский край,"край Приморский, г Владивосток, ул Прапорщика ...",CЗ АЗИЯ-КОНТРАКТ,8002.40,120,25,131687.0,Владивосток
315,26213,Приморский край,край Приморский г Владивосток ул Светланская д...,АТЛАНТИКС,0.00,0,5,31705.0,Владивосток
316,26212,Приморский край,край Приморский г Владивосток ул Светланская д...,АТЛАНТИКС,25078.00,176,24,49695.0,Владивосток
727,37898,Приморский край,край Приморский город Владивосток улица Енисей...,ДВ КОНСТРАКШЕН,9247.28,322,25,121003.0,Владивосток
728,37899,Приморский край,край Приморский город Владивосток улица Енисей...,ДВ КОНСТРАКШЕН,7249.87,252,19,123194.0,Владивосток
...,...,...,...,...,...,...,...,...,...
8693,41419,Приморский край,"Приморский край, г Владивосток, улица Басаргин...",СЗПАСИФИКБЭЙ,0.00,0,4,NaN,Владивосток
9061,45858,Приморский край,"край Приморский, город Владивосток, улица Держ...",СИК АТЛАНТ,817.70,9,2,NaN,Владивосток
9480,37348,Приморский край,край Приморский г Владивосток пр-кт Океанский ...,СТРОЙЗАКАЗ,14572.01,450,29,121725.0,Владивосток
9731,45880,Приморский край,"край Приморский, город Уссурийск, улица Новони...",УСИ,2232.93,44,5,NaN,Уссурийск


In [81]:
# Сравниваем среднюю цену квадратного метра в городах Приморского края
data[data['Регион'] == 'Приморский край'].groupby('Населенный пункт')['Цена за 1 кв.м'].mean()

Населенный пункт
Артем           89952.000000
Владивосток    118553.072727
Находка                  NaN
Тавричанка      85100.000000
Уссурийск      100706.222222
Name: Цена за 1 кв.м, dtype: float64

In [37]:
# количество строящихся объектов в регионах
data.groupby('Регион')['ID дома'].count()

Регион
Адыгея Республика                   25
Алтай Республика                    16
Алтайский край                      94
Амурская область                    30
Архангельская область               55
                                  ... 
Челябинская область                173
Чеченская Республика                 3
Чувашская Республика                69
Ямало-Ненецкий Автономный округ     24
Ярославская область                 98
Name: ID дома, Length: 81, dtype: int64

In [38]:
# количество строящихся квартир в регионах
data.groupby('Регион')['Кол-во квартир'].sum()

Регион
Адыгея Республика                   2345
Алтай Республика                    1435
Алтайский край                     15830
Амурская область                    3285
Архангельская область               8896
                                   ...  
Челябинская область                24302
Чеченская Республика                 788
Чувашская Республика               10853
Ямало-Ненецкий Автономный округ     1817
Ярославская область                14530
Name: Кол-во квартир, Length: 81, dtype: int64

In [39]:
# Объем строящегося жилья в квадратных метрах по всей стране
data['Жилая площадь'].sum()

95521044.57

In [40]:
# Объем строящегося жилья в квадратных метрах по регионам
data.groupby('Регион')['Жилая площадь'].sum()

Регион
Адыгея Республика                   106878.82
Алтай Республика                     74649.50
Алтайский край                      804492.88
Амурская область                    169980.43
Архангельская область               394446.03
                                      ...    
Челябинская область                1239850.40
Чеченская Республика                 70559.48
Чувашская Республика                624859.40
Ямало-Ненецкий Автономный округ      87249.23
Ярославская область                 775742.32
Name: Жилая площадь, Length: 81, dtype: float64